In [1]:
from pyspark.sql import SparkSession, SQLContext
from pyspark import SparkConf,SparkContext

In [1]:
cwd = os.getcwd()
for part in cwd.split('/'):
    if part.lower().startswith('edureka'):
        user_id = part.title()
user_id

'Edureka_121039'

In [2]:
app_name = '{0} : Decision Trees'.format(user_id)
app_name

'Edureka_121039 : Decision Trees'

In [3]:
spark = SparkSession.builder.appName(app_name).getOrCreate()
sparkContext = spark.sparkContext
sqlContext = SQLContext(sparkContext)

In [5]:
def get_hdfs_filepath(file_name):
    my_hdfs = '/user/{0}/datasets'.format(user_id.lower())
    return os.path.join(my_hdfs, file_name)

### Dataset
Details can be referred [here](http://archive.ics.uci.edu/ml/datasets/Bike+Sharing+Dataset "Bike Sharing dataset")

In [6]:
BIKE_SHARING_CSV = get_hdfs_filepath('bikeSharing.csv')

In [7]:
df = spark.read.csv(BIKE_SHARING_CSV,inferSchema=True,header=True)

In [8]:
df.cache()

DataFrame[instant: int, dteday: timestamp, season: int, yr: int, mnth: int, hr: int, holiday: int, weekday: int, workingday: int, weathersit: int, temp: double, atemp: double, hum: double, windspeed: double, casual: int, registered: int, cnt: int]

In [9]:
df.show()

+-------+--------------------+------+---+----+---+-------+-------+----------+----------+----+------+----+---------+------+----------+---+
|instant|              dteday|season| yr|mnth| hr|holiday|weekday|workingday|weathersit|temp| atemp| hum|windspeed|casual|registered|cnt|
+-------+--------------------+------+---+----+---+-------+-------+----------+----------+----+------+----+---------+------+----------+---+
|      1|2011-01-01 00:00:...|     1|  0|   1|  0|      0|      6|         0|         1|0.24|0.2879|0.81|      0.0|     3|        13| 16|
|      2|2011-01-01 00:00:...|     1|  0|   1|  1|      0|      6|         0|         1|0.22|0.2727| 0.8|      0.0|     8|        32| 40|
|      3|2011-01-01 00:00:...|     1|  0|   1|  2|      0|      6|         0|         1|0.22|0.2727| 0.8|      0.0|     5|        27| 32|
|      4|2011-01-01 00:00:...|     1|  0|   1|  3|      0|      6|         0|         1|0.24|0.2879|0.75|      0.0|     3|        10| 13|
|      5|2011-01-01 00:00:...|    

In [10]:
print("Our dataset has %d rows." % df.count())

Our dataset has 1000 rows.


In [11]:
df = df.drop("instant").drop("dteday").drop("casual").drop("registered")
display(df)

DataFrame[season: int, yr: int, mnth: int, hr: int, holiday: int, weekday: int, workingday: int, weathersit: int, temp: double, atemp: double, hum: double, windspeed: double, cnt: int]

In [12]:
df.show()

+------+---+----+---+-------+-------+----------+----------+----+------+----+---------+---+
|season| yr|mnth| hr|holiday|weekday|workingday|weathersit|temp| atemp| hum|windspeed|cnt|
+------+---+----+---+-------+-------+----------+----------+----+------+----+---------+---+
|     1|  0|   1|  0|      0|      6|         0|         1|0.24|0.2879|0.81|      0.0| 16|
|     1|  0|   1|  1|      0|      6|         0|         1|0.22|0.2727| 0.8|      0.0| 40|
|     1|  0|   1|  2|      0|      6|         0|         1|0.22|0.2727| 0.8|      0.0| 32|
|     1|  0|   1|  3|      0|      6|         0|         1|0.24|0.2879|0.75|      0.0| 13|
|     1|  0|   1|  4|      0|      6|         0|         1|0.24|0.2879|0.75|      0.0|  1|
|     1|  0|   1|  5|      0|      6|         0|         2|0.24|0.2576|0.75|   0.0896|  1|
|     1|  0|   1|  6|      0|      6|         0|         1|0.22|0.2727| 0.8|      0.0|  2|
|     1|  0|   1|  7|      0|      6|         0|         1| 0.2|0.2576|0.86|      0.0|  3|

In [13]:
df.printSchema()

root
 |-- season: integer (nullable = true)
 |-- yr: integer (nullable = true)
 |-- mnth: integer (nullable = true)
 |-- hr: integer (nullable = true)
 |-- holiday: integer (nullable = true)
 |-- weekday: integer (nullable = true)
 |-- workingday: integer (nullable = true)
 |-- weathersit: integer (nullable = true)
 |-- temp: double (nullable = true)
 |-- atemp: double (nullable = true)
 |-- hum: double (nullable = true)
 |-- windspeed: double (nullable = true)
 |-- cnt: integer (nullable = true)



In [14]:
from pyspark.sql.functions import col  # for indicating a column using a string in the line below

In [15]:
df = df.select([col(c).cast("double").alias(c) for c in df.columns])
df.printSchema()

root
 |-- season: double (nullable = true)
 |-- yr: double (nullable = true)
 |-- mnth: double (nullable = true)
 |-- hr: double (nullable = true)
 |-- holiday: double (nullable = true)
 |-- weekday: double (nullable = true)
 |-- workingday: double (nullable = true)
 |-- weathersit: double (nullable = true)
 |-- temp: double (nullable = true)
 |-- atemp: double (nullable = true)
 |-- hum: double (nullable = true)
 |-- windspeed: double (nullable = true)
 |-- cnt: double (nullable = true)



In [16]:
train, test = df.randomSplit([0.7, 0.3])

In [17]:
print("We have %d training examples and %d test examples." % (train.count(), test.count()))

We have 707 training examples and 293 test examples.


In [18]:
display(train.select("hr", "cnt"))

DataFrame[hr: double, cnt: double]

In [22]:
from pyspark.ml.feature import VectorAssembler, VectorIndexer

In [23]:
featuresCols = df.columns
featuresCols.remove('cnt')
# This concatenates all feature columns into a single feature vector in a new column "rawFeatures".
vectorAssembler = VectorAssembler(inputCols=featuresCols, outputCol="rawFeatures")
# This identifies categorical features and indexes them.
vectorIndexer = VectorIndexer(inputCol="rawFeatures", outputCol="features", maxCategories=4)

In [24]:
from pyspark.ml.regression import GBTRegressor
# Takes the "features" column and learns to predict "cnt"
gbt = GBTRegressor(labelCol="cnt")

In [25]:
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.evaluation import RegressionEvaluator

In [26]:
# Define a grid of hyperparameters to test:
#  - maxDepth: max depth of each decision tree in the GBT ensemble
#  - maxIter: iterations, i.e., number of trees in each GBT ensemble
# In this example notebook, we keep these values small. 
# In practice, to get the highest accuracy, you would likely want to try deeper trees (10 or higher) and 
# more trees in the ensemble (>100).
paramGrid = ParamGridBuilder().addGrid(gbt.maxDepth, [2, 5]).addGrid(gbt.maxIter, [10, 100]).build()
# We define an evaluation metric.  This tells CrossValidator how well we are doing by comparing the true labels with predictions.
evaluator = RegressionEvaluator(metricName="rmse", labelCol=gbt.getLabelCol(), predictionCol=gbt.getPredictionCol())
# Declare the CrossValidator, which runs model tuning for us.
cv = CrossValidator(estimator=gbt, evaluator=evaluator, estimatorParamMaps=paramGrid)

In [27]:
from pyspark.ml import Pipeline
pipeline = Pipeline(stages=[vectorAssembler, vectorIndexer, cv])

In [ ]:
pipelineModel = pipeline.fit(train)

In [31]:
predictions = pipelineModel.transform(test)

In [32]:
display(predictions.select("cnt", "prediction", *featuresCols))
predictions.select("cnt", "prediction", *featuresCols).show()

DataFrame[cnt: double, prediction: double, season: double, yr: double, mnth: double, hr: double, holiday: double, weekday: double, workingday: double, weathersit: double, temp: double, atemp: double, hum: double, windspeed: double]

+----+------------------+------+---+----+---+-------+-------+----------+----------+----+------+----+---------+
| cnt|        prediction|season| yr|mnth| hr|holiday|weekday|workingday|weathersit|temp| atemp| hum|windspeed|
+----+------------------+------+---+----+---+-------+-------+----------+----------+----+------+----+---------+
|22.0| 24.28854670989856|   1.0|0.0| 1.0|0.0|    0.0|    0.0|       0.0|       1.0|0.04|0.0758|0.57|   0.1045|
|33.0|26.401313942358094|   1.0|0.0| 1.0|0.0|    0.0|    0.0|       0.0|       1.0|0.16|0.1818| 0.8|   0.1045|
| 5.0|12.174393428860355|   1.0|0.0| 1.0|0.0|    0.0|    1.0|       1.0|       1.0|0.12|0.1212| 0.5|   0.2836|
| 5.0|10.072339913508364|   1.0|0.0| 1.0|0.0|    0.0|    2.0|       1.0|       1.0|0.16|0.1818|0.55|   0.1045|
| 7.0| 3.074259985483318|   1.0|0.0| 1.0|0.0|    0.0|    3.0|       1.0|       2.0|0.16| 0.197|0.86|   0.0896|
| 3.0| 8.958387747728121|   1.0|0.0| 1.0|0.0|    0.0|    3.0|       1.0|       2.0|0.22|0.2727|0.93|      0.0|
|

In [ ]:
rmse = evaluator.evaluate(predictions)
print("RMSE on our test set: %g" , rmse)

In [ ]:
display(predictions.select("hr", "prediction"))
predictions.select("hr", "prediction").show()

In [29]:
import matplotlib.pyplot as plt

In [30]:
display(predictions.select("hr", "prediction"))

DataFrame[hr: double, prediction: double]

In [31]:
plt.show()

In [1]:
spark.stop()

NameError: name 'spark' is not defined